#Top five reinsurance companies financial data analysis

This colab notebook serves as a machine for ingesting and transforming publicly available financial data from the top five reinsurance companies. The main aim is to clean each data set, bring all together and finally visualise the numbers to conduct a peer analysis. At the same time the data must be stored in a format which enables an easy and automated update for future financial data releases.

Although the files are available on each companies investor relations website, I downloaded it and stored it in a [github repository](https://github.com/lukasbelka/Reinsurers-financial-data-analysis) together with this notebook. The repository also contains [the links](https://github.com/lukasbelka/Reinsurers-financial-data-analysis/blob/main/raw_financial_data/readme.md) to the companies websites.

## Data Ingestion, exploration and cleaning

The first step is to import the relevant packages and load the financial data from the repository.

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
url = ('https://github.com/lukasbelka/Reinsurers-financial-data-analysis/blob/main/raw_financial_data/HNR/HNR_AR2021_XLS_en.xlsx?raw=true')
xls_21 = pd.read_excel(url, sheet_name=None)

# Let's see which sheets the excel file contains
print(xls_21.keys())    

dict_keys(['Table of content', 'Key figures', 'Remuneration Executive Board', 'Remuneration Supervisory Board', 'Consolidated balance sheet', 'Consolid. statement of income', 'Consolid. comprehensive income', "Consolid. shareholders' equity", 'Consolid. cash flow statement', 'Segment report - balance sheet', 'Segment report - income statem.', 'Investments by currencies', 'Gross written premium', 'Investment income', 'Net loss reserve P&C', 'Gains&losses on investments'])


As a first step I will load the income statements sheet and have a quick look at the raw data.

In [48]:

# The data frame from the income statements sheet will be called finally 'is_hr' after the cleanup.

is_hr_2021_raw = pd.read_excel(url, sheet_name='Consolid. statement of income', skiprows=2, header=1)
print(is_hr_2021_raw.info())
print(is_hr_2021_raw.head())
print(is_hr_2021_raw)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   In EUR thousand      37 non-null     object 
 1   Notes                20 non-null     float64
 2   1.1. - 31.12.2021    34 non-null     float64
 3   1.1. - 31.12.2020¹   34 non-null     float64
dtypes: float64(3), object(1)
memory usage: 1.3+ KB
None
                    In EUR thousand  Notes  1.1. - 31.12.2021  \
0             Gross written premium    7.1         27762314.0   
1             Ceded written premium    NaN          2905054.0   
2  Change in gross unearned premium    NaN          -737631.0   
3  Change in ceded unearned premium    NaN            24023.0   
4                Net premium earned    NaN         24143652.0   

   1.1. - 31.12.2020¹   
0           24770342.0  
1            2442720.0  
2           -1028172.0  
3              61345.0  
4           21360795.0  


We have the following observations:
- Numbers in EUR thousand
- Four columns in the consolidated income statement.
  - The first one shows the names of all income statement positions
  - The second one seems to contain notes references
  - The third and fourth hold the figures for the financial years 2020 and 2021.
- We have some NaN (**N**ot **a** **N**umber) values.

Let's deal with the data as follows:
- Skip the first two rows of the sheet to obtain the third one as the header (done already while creating the data frame)
- Clean the data by dropping all NaN-rows in order to get rid of empty cells
- Delete the second column with notes references
- Include a column with the company name (which enables us to identify the positions (rows) after combining the data from all reinsurers

Here we go!

In [46]:
is_hr_2021_drop_c = is_hr_2021_raw.drop(columns='Notes')
is_hr_2021_drop_c.drop(is_hr_2021_drop_c.tail(3).index, inplace=True)
is_hr_final = is_hr_2021_drop_c.rename(columns={"1.1. - 31.12.2021": "2021", "1.1. - 31.12.2020¹ ": "2020"})

print(is_hr_final)

                                      In EUR thousand         2021  \
0                               Gross written premium  27762314.00   
1                               Ceded written premium   2905054.00   
2                    Change in gross unearned premium   -737631.00   
3                    Change in ceded unearned premium     24023.00   
4                                  Net premium earned  24143652.00   
5                          Ordinary investment income   1555591.00   
6   Profit/loss from investments in associated com...     35743.00   
7            Realised gains and losses on investments    281026.00   
8       Change in fair value of financial instruments     36114.00   
9   Total depreciation, impairments and appreciati...     87665.00   
10                          Other investment expenses    146047.00   
11   Net income from investments under own management   1674762.00   
12  Income/expense on funds withheld and contract ...    268250.00   
13                  